In [4]:
import pandas as pd     
from  sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import curve_fit
import numpy as np

In [25]:
height_biomass_df = pd.read_csv('H_AGB_NFI4_PNOA.csv')

p99_height_max = np.percentile(height_biomass_df['Hmax'],99)
p01_height_max = np.percentile(height_biomass_df['Hmax'],1)

height_biomass_df = height_biomass_df[(height_biomass_df['Hmax']<p99_height_max) & (height_biomass_df['Hmax']>p01_height_max) ]

In [75]:
def power_law(x,a,b):
    return a*np.power(x,b)

def exponential(x,a,b):
    return np.exp(a+x*b)    


fit_table = pd.DataFrame()
func_dict = {"power_law" : power_law, "exponential" : exponential}
metric_dict = {"R2" : r2_score, "MAE" : mean_absolute_error}

# for dominant_type in height_biomass_df.Type.unique():

#     df2 = height_biomass_df[ height_biomass_df.Type == dominant_type ]

# for biome in height_biomass_df.Biome.unique():

#     df = height_biomass_df[ height_biomass_df.Biome == biome ] 

df = height_biomass_df
for covar in df.drop(columns=["Biome","Type","AGB"]).columns:

    for func in func_dict:
        try :
            fit_result = curve_fit(func_dict[func],df[covar],df.AGB,method='trf',**{'loss':'huber'})
            a=fit_result[0][0]
            b=fit_result[0][1]

            agb_estimate = func_dict[func](df[covar],a,b)
            
            for metric in metric_dict:
                score = metric_dict[metric](df.AGB,agb_estimate)

                results = {
                    "Height" : [covar],
                    "Function" : [func],
                    "a" : [a],
                    "b" : [b],
                    "Metric" : [metric],
                    "Score" : [score], 
                    "Type" : [dominant_type],
                    "Biome" : [biome]

                }
                
                this_row = pd.DataFrame.from_dict(results)
                fit_table = pd.concat([fit_table,this_row], axis='rows')
        except:
            continue       
        
fit_table[ (fit_table.Metric == 'MAE') & (fit_table.Biome != "Temperate Broadleaf & Mixed Forests") ]


,Height,Function,a,b,Metric,Score,Type,Biome
0,Hp10,exponential,3.859283,0.097858,MAE,44.122702,Gymnosperm,"Mediterranean Forests, Woodlands & Scrub"
0,Hp20,exponential,3.767666,0.094914,MAE,43.144548,Gymnosperm,"Mediterranean Forests, Woodlands & Scrub"
0,Hp30,power_law,33.717197,0.495101,MAE,42.769476,Gymnosperm,"Mediterranean Forests, Woodlands & Scrub"
0,Hp30,exponential,3.676029,0.093901,MAE,42.438492,Gymnosperm,"Mediterranean Forests, Woodlands & Scrub"
0,Hp40,power_law,26.622438,0.575892,MAE,42.071315,Gymnosperm,"Mediterranean Forests, Woodlands & Scrub"
0,Hp40,exponential,3.605930,0.091935,MAE,41.857919,Gymnosperm,"Mediterranean Forests, Woodlands & Scrub"
0,Hp50,power_law,20.893447,0.653961,MAE,41.451260,Gymnosperm,"Mediterranean Forests, Woodlands & Scrub"
0,Hp50,exponential,3.533601,0.090923,MAE,41.373039,Gymnosperm,"Mediterranean Forests, Woodlands & Scrub"
0,Hp60,power_law,16.363846,0.730543,MAE,40.837526,Gymnosperm,"Mediterranean Forests, Woodlands & Scrub"
0,Hp60,exponential,3.468490,0.089522,MAE,40.962588,Gymnosperm,"Mediterranean Forests, Woodlands & Scrub"
